In [1]:
# Bibliotecas

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import math
import win32com.client #Executar no prompt Anaconda: pip install pywin32
from geographiclib.geodesic import Geodesic #Executar no prompt Anaconda: pip install geographiclib

In [2]:
# Carregando lista de Portos

Portos = pd.read_excel('TCC.xlsx', sheet_name = 'Portos', nrows = 10)

In [3]:
Portos

,Portos,Latitude,Longitude
0,PORTO DO ACU,-21.843250,-41.00089
1,ANGRA DOS REIS,-23.011830,-44.31065
2,CABEDELO,-6.971611,-34.83998
3,FORTALEZA,-3.714480,-38.49648
4,TMIB,-10.845280,-36.91853
5,PECEM,-3.532126,-38.80311
6,RIO DE JANEIRO,-22.849820,-43.17784
7,SANTOS,-23.951090,-46.35358
8,SAO SEBASTIAO,-23.795470,-45.39520
9,SUAPE,-8.391142,-34.96811


In [4]:
# Carregando lista de Plataformas

Plataformas = pd.read_excel('TCC.xlsx', sheet_name = 'Plataformas', nrows = 10)

In [5]:
Plataformas

,Plataformas,Latitude,Longitude
0,Plataforma 01,"-24,109335","-39,813312"
1,Plataforma 02,"-31,36303","-40,61051"
2,Plataforma 03,"-26,244168","-45,708166"
3,Plataforma 04,"-24,75682","-47,015539"
4,Plataforma 05,"-24,468399","-43,202861"
5,Plataforma 06,"-23,812492","-43,288848"
6,Plataforma 07,"-16,207462","-33,628008"
7,Plataforma 08,"-10,977465","-28,767821"
8,Plataforma 09,"-3,335002","-28,780897"
9,Plataforma 10,"-24,163783","-32,794469"


In [6]:
# Instancia Excel
File = win32com.client.Dispatch("Excel.Application")

File.Visible = 1

# Abre o arquivo
Workbook = File.Workbooks.open("C:\\Users\\Thais\\Documents\\BI Master\\TCC - BI Master\\TCC.xlsx")

In [7]:
# Inicializando o Chrome com definição dos parâmetros

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
wd = webdriver.Chrome(r"chromedriver.exe",options=chrome_options)

In [8]:
# Limpando cookies

wd.delete_all_cookies()

In [9]:
# Abrindo o site do Marine Traffic

wd.get("https://www.marinetraffic.com/")
time.sleep(4)

In [10]:
# Clica em "Agree" (aceitando política de privacidade do Site)

wd.find_element_by_xpath('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()

In [11]:
# Clica em "Search MarineTraffic" para desativar mensagem de dica

wd.find_element_by_id('searchMarineTraffic').click()
time.sleep(1)
for i in range(2,5):
    try:
        wd.find_element_by_xpath('/html/body/div[' + str(i) + ']/div[3]/div/div[2]/div[2]/div/button').click()
    except Exception as e:
        erro = str(e)
        #print(erro)

In [12]:
# Scrapping

#Mudando para a aba Portos do excel
File.Sheets(1).Activate()

linha = Portos['Portos']

for l in range(len(linha)):
    for i in range(0,2):
        try:
            if (l > 0) or (i > 0):
                wd.find_element_by_id('searchMarineTraffic').click()
                time.sleep(2)
            
            # Clica no botão "Search Results"
            for i in range(2,5):
                try:
                    wd.find_element_by_xpath('/html/body/div[' + str(i) + ']/div[3]/div/div[2]/div/div/button[1]').click()
                except Exception as e:
                    erro = str(e)
                    #print(erro)
            time.sleep(2)
            
            # Clica no filtro "Ports"
            for i in range(2,5):
                try:
                    wd.find_element_by_xpath('/html/body/div[' + str(i) + ']/div[3]/div/div[4]/div/div[3]/span').click()
                except Exception as e:
                    erro = str(e)
                    #print(erro)
            time.sleep(2)
            
            # Preenche campo de pesquisa (Porto)
            porto = wd.find_element_by_id('searchMT')
            porto.clear()
            porto.send_keys(linha[l])
            time.sleep(3)
            
            # Seleciona o Porto
            for i in range(2,5):
                try:
                    wd.find_element_by_xpath('/html/body/div[' + str(i) + ']/div[3]/div/div[3]/div[1]/div[1]/a').click()
                except Exception as e:
                    erro = str(e)
                    #print(erro)
            time.sleep(5)
            
            # Recupera Latitude / Longitude
            latLong = wd.find_element_by_xpath('//*[@id="generalInfo"]/div[2]/div/div/div/div/p[1]/b/a').text
            #print(latLong)
            latitude = latLong.split('/', 1)[0].replace('°', '').strip()
            longitude = latLong.split('/', 1)[1].replace('°', '').strip()
            
            # Grava no excel a Latitude / Longitude do Porto
            File.Sheets(1).Cells(l+2, 2).Value = latitude
            File.Sheets(1).Cells(l+2, 3).Value = longitude
            
            erro = None
            break
        except Exception as e:
            erro = str(e)
            #print(linha[l], ': ', erro)

        if erro:
            time.sleep(2)

In [13]:
#Calculando as distâncias entre os Portos e Plataformas

#Mudando para a aba Distancias do excel
File.Sheets(4).Activate()

linha = Portos['Portos']
coluna = Plataformas['Plataformas']
Pi = 3.14159

for l in range(len(linha)):
    Porto = File.Sheets(4).Cells(l+2,1).Value
    latitudePorto = File.Sheets(1).Cells(l+2,2).Value
    longitudePorto = File.Sheets(1).Cells(l+2,3).Value
    for c in range(len(coluna)):
        Plataforma = File.Sheets(4).Cells(1,c+2).Value
        latitudePlataforma = File.Sheets(3).Cells(c+2,2).Value
        latitudePlataforma = float(latitudePlataforma.replace(',','.'))
        longitudePlataforma = File.Sheets(3).Cells(c+2,3).Value
        longitudePlataforma = float(longitudePlataforma.replace(',','.'))
        
        if (abs(abs(latitudePlataforma) - abs(latitudePorto)) > 7) or ((abs(longitudePlataforma) - abs(longitudePorto)) > 2):
            d = Geodesic.WGS84.Inverse(latitudePorto, longitudePorto, latitudePorto - 2, longitudePorto + 7)
            mn = d['s12'] * 0.000539957
            
            if (abs(abs(latitudePlataforma) - abs(latitudePorto - 2)) > 12):
                d = Geodesic.WGS84.Inverse(latitudePorto - 2, longitudePorto + 7, latitudePlataforma, longitudePorto + 7)
                mn += d['s12'] * 0.000539957
                
                d = Geodesic.WGS84.Inverse(latitudePlataforma, longitudePorto + 7, latitudePlataforma, longitudePlataforma)
                mn += d['s12'] * 0.000539957
            else:
                d = Geodesic.WGS84.Inverse(latitudePorto - 2, longitudePorto + 7, latitudePlataforma, longitudePlataforma)
                mn += d['s12'] * 0.000539957
        else:
            d = Geodesic.WGS84.Inverse(latitudePorto, longitudePorto, latitudePlataforma, longitudePlataforma)
            mn = d['s12'] * 0.000539957
        
        File.Sheets(4).Cells(l+2,c+2).Value = mn
time.sleep(3)

In [14]:
# Salve o excel
Workbook.Save()

In [15]:
Portos = pd.read_excel('TCC.xlsx', sheet_name = 'Portos', nrows = 10)
Portos

,Portos,Latitude,Longitude
0,PORTO DO ACU,-21.843250,-41.00089
1,ANGRA DOS REIS,-23.011830,-44.31065
2,CABEDELO,-6.971611,-34.83998
3,FORTALEZA,-3.714480,-38.49648
4,TMIB,-10.845280,-36.91853
5,PECEM,-3.532126,-38.80311
6,RIO DE JANEIRO,-22.849820,-43.17784
7,SANTOS,-23.951090,-46.35358
8,SAO SEBASTIAO,-23.795470,-45.39520
9,SUAPE,-8.391142,-34.96811


In [16]:
Distancias = pd.read_excel('TCC.xlsx', sheet_name = 'Distancias', nrows = 10, header = 0, index_col = 0)
Distancias

,Plataforma 01,Plataforma 02,Plataforma 03,Plataforma 04,Plataforma 05,Plataforma 06,Plataforma 07,Plataforma 08,Plataforma 09,Plataforma 10
De \ Para,,,,,,,,,,
PORTO DO ACU,150.612545,977.064371,1059.518210,1121.096441,912.149089,916.784818,537.494945,1483.601471,1944.382935,474.894890
ANGRA DOS REIS,256.447291,820.978511,207.869515,932.262055,106.331802,73.975469,728.594346,1745.563359,2209.638501,638.256237
CABEDELO,1995.126697,2427.757573,2428.908835,2423.192830,2200.067787,2169.885908,982.982207,432.624769,422.708264,1613.154080
FORTALEZA,1991.560281,2437.175870,2429.059185,2420.923238,2197.174676,2165.961047,1075.183512,789.685337,583.320893,1609.553312
TMIB,1306.401913,2085.097078,2080.982366,1632.368033,1454.677696,1433.482471,373.592426,481.171283,1000.746672,1124.779758
PECEM,1985.726625,2432.428489,2423.533617,2415.194819,2191.395863,2160.095929,1082.721789,808.012376,601.450510,1603.719776
RIO DE JANEIRO,200.280642,858.200325,926.832414,994.674854,96.804169,57.892777,670.825127,1669.484587,2132.431132,577.854486
SANTOS,359.340823,730.534047,141.589026,60.312052,175.566644,168.735019,1064.898101,1919.581235,2385.695974,744.445936
SAO SEBASTIAO,307.343554,753.506774,147.453103,105.814109,126.877088,115.904333,1032.811545,1854.177509,2319.334690,692.530144


In [17]:
# Fecha o excel
File.Quit()

# Encerra o Scrapping
wd.quit()